# Step5 演習07 LSTMとGRU
---

本演習では、LSTMとGRUをスクラッチで実装します。

**はじめに**
- for文やwhile文の利用は明示的な利用指示がない場所での利用は避けてください。

**本演習の目的**
- LSTMとGRUのパラメータの数を理解する。
- LSTMとGRUの順伝播を実装する。
- Gradient Checkのコードを実装する。

## ライブラリのインストール

まずはじめに、本演習で利用するライブラリのインポートを行います。

- [numpy](http://www.numpy.org) 数値計算を行うための基本パッケージの公式ドキュメント

In [1]:
import numpy as np

## LSTMとGRU両方で使う関数の準備

いつもどおりglorot uniformで初期化します。

In [2]:
def glorot_uniform(shape, seed=1234):
    rng = np.random.RandomState(seed)
    if len(shape) == 2:
        fan_in, fan_out = shape
        std = np.sqrt(6./(fan_in+fan_out))
        return rng.uniform(low=-std, high=std, size=(fan_in, fan_out))
    else:
        raise NotImplementedError()

シグモイド関数とハイパボリックタンジェント関数を用意します。また、それぞれを微分した関数も用意します。ここで、`dsigmoid`や`dtanh`の引数はは既に`sigmoid`や`tanh`を計算した値とします。なので、ニューラルネットワークの章の演習とは少し異なります。

In [3]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def dsigmoid(sigmoid_x):
    return sigmoid_x * (1 - sigmoid_x)

def tanh(x):
    return np.tanh(x)

def dtanh(tanh_x):
    return (1-tanh_x**2)

## LSTMの実装

ではここからLSTMの実装を行います。この演習では入力、出力、忘却ゲートはありますが、覗き穴結合はないタイプのLSTM（下図参照）を実装します。

<img src="./img/step5_LSTM.png" width="360" height="120">


### LSTMのパラメータの初期化

覗き穴結合がない場合、入力ゲート$\boldsymbol{i}(t)$、出力ゲート$\boldsymbol{o}(t)$、忘却ゲート$\boldsymbol{f}(t)$そして、活性化関数$f$後の値$\boldsymbol{a}(t)$は以下のようにして求められます。

$$
\boldsymbol{i}(t)
=
\sigma(\boldsymbol{W_i} \boldsymbol{x}(t) + \boldsymbol{U_i} \boldsymbol{h}(t-1) + \boldsymbol{b}_i)\tag{1.1}
$$

$$
\boldsymbol{o}(t)
=
\sigma(\boldsymbol{W_o} \boldsymbol{x}(t) + \boldsymbol{U_o} \boldsymbol{h}(t-1) + \boldsymbol{b}_o)\tag{1.2}
$$


$$
\boldsymbol{f}(t)
=
\sigma(\boldsymbol{W_f} \boldsymbol{x}(t) + \boldsymbol{U_f} \boldsymbol{h}(t-1) + \boldsymbol{b}_f)\tag{1.3}
$$

$$
\boldsymbol{a}(t)
=
f(\boldsymbol{W_a} \boldsymbol{x}(t) + \boldsymbol{U_a} \boldsymbol{h}(t-1) + \boldsymbol{b}_a)\tag{1.4}
$$

ここで、これら４式の活性化関数内の部分をそれぞれ$\boldsymbol{\hat{i}}(t)$、$\boldsymbol{\hat{o}}(t)$、$\boldsymbol{\hat{f}}(t)$、$\boldsymbol{\hat{a}}(t)$、と置きます。なので例えば、$\boldsymbol{i}(t)=\sigma(\boldsymbol{\hat{i}}(t))$となります。すると、これを用いると、

$$
\left(
\begin{array}{c}
    \boldsymbol{\hat{i}}(t)\\
    \boldsymbol{\hat{o}}(t)\\
    \boldsymbol{\hat{f}}(t)\\
    \boldsymbol{\hat{a}}(t)
\end{array}
\right)
=
\left(
\begin{array}{ccc}
    \boldsymbol{W_i} & \boldsymbol{U_i} & \boldsymbol{b_i}\\
    \boldsymbol{W_o} & \boldsymbol{U_o} & \boldsymbol{b_o}\\
    \boldsymbol{W_f} & \boldsymbol{U_f} & \boldsymbol{b_f}\\
    \boldsymbol{W_a} & \boldsymbol{U_a} & \boldsymbol{b_a}\\
\end{array}
\right)
\left(
\begin{array}{c}
    \boldsymbol{x}(t)\\
    \boldsymbol{h}(t-1)\\
    \boldsymbol{1}
\end{array}
\right)\tag{1.5}
$$

と書くことができます。更にこれを

$$
\boldsymbol{\widehat{gates}}(t) = \boldsymbol{\widehat{W}}\boldsymbol{\widehat{x}}(t) \tag{1.6}
$$
のようにまとめて書くことにします。

**【課題１】**　LSTMのパラメータを初期化する関数を用意します。

前置きが長くなりましたが、numpyで計算するときもですし、GPU上で計算するときもこのようにまとめることにより効率よく計算できます。なので、LSTMは１２個パラーメータ（重み８個、バイアス４個）が存在しますが、式(1.6)に従って１つのように扱います。

関数`LSTM_init_parameters`は引数として、
- n_input: 入力の次元
- n_hidden: 隠れ層のユニット数

があります。この関数内では式(1.6)に従って`W`を初期化します。バイアスは０で初期化しますが、まずは`glorot_uniform`で一括でランダムに初期化した後にバイアスの箇所のみ0に置き換えます。

In [4]:
#Coursedele-02 Step5 QuestionNumber20 0849a33a688d3113abe78ba50ab337e3
def LSTM_init_parameters(n_input, n_hidden):
    ###############START CODE HERE###############
    # 式(1.6)に従って重みをすべてまとめて初期化
    W = glorot_uniform(shape = (n_input + n_hidden + 1, n_hidden * 4))
    ################END CODE HERE################
    W[-1] = 0 #バイアスはゼロに初期化
    return W

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [5]:
%%bash
./validation_client.py dele-02 5 20 Step5_07.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### LSTMの順伝播の実装

**【課題２】**　LSTMの順伝播を実装します。

LSTMの順伝播は式(1.1)~(1.6)に加えて下式を計算します。

$
\boldsymbol{c}(t) = \boldsymbol{i}(t)\odot \boldsymbol{a}(t)+\boldsymbol{f}(t)\odot \boldsymbol{c}(t-1)\tag{1.7}
$


$
\boldsymbol{h}(t) = \boldsymbol{o}(t)\odot g(\boldsymbol{c}(t))\tag{1.8}
$

関数`LSTM_forward`は４つの引数があります。
- X: 入力（データ長、ミニバッチサイズ、入力サイズ）
- W: `LSTM_init_parameters`から得られる重み
- c0, h0: LSTMのCECと隠れ層の値（なければゼロで初期化）

ここで実装していただく箇所の説明をします。
- `ones`: 式(1.5)の最後の1を作成します。形は(ミニバッチサイズ、１)です。
- `x_h`: 式(1.6)の$\hat{\boldsymbol{x}}(t)$を作成します。ここでは[np.hstack](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.hstack.html)を用います。[np.concatenate](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.concatenate.html)でも可能ですがこの際はaxisの指定が必要です。
- `gates_h`: 式(1.6)の$\boldsymbol{\widehat{gates}}(t)$を計算します。
- `i_h, o_h, f_h, a_h`: [np.split](https://docs.scipy.org/doc/numpy/reference/generated/numpy.split.html)を用いて、$\boldsymbol{\widehat{gates}}(t)$を分離します。
- `i, o, f, a`: 式(1.1)~(1.4)を計算します。
- `c`: 式(1.7)を計算します。
- `ct`: 式(1.8)の$g(\boldsymbol{c}(t))$の箇所をまず計算します。
- `h`: 式(1.8)を計算します。


In [6]:
#Coursedele-02 Step5 QuestionNumber21 32a83f2a4e56fe6eb6d3d4614764ad63
def LSTM_forward(X, W, c0 = None, h0 = None):
    # パラメータを取得
    length, batch_size, n_input = X.shape
    n_hidden = W.shape[1]//4
    n_input = W.shape[0] - n_hidden - 1
    
    # c0, h0の指定がない場合ゼロで初期化
    if c0 is None: c0 = np.zeros((batch_size, n_hidden))
    if h0 is None: h0 = np.zeros((batch_size, n_hidden))
    
    # 各ステップの状態を保存しておくための配列
    X_h = np.zeros((length, batch_size, W.shape[0]))
    H = np.zeros((length, batch_size, n_hidden))
    Gates = np.zeros((length, batch_size, n_hidden*4))
    C = np.zeros((length, batch_size, n_hidden))
    Ct = np.zeros((length, batch_size, n_hidden))
    
    for t in range(length):
        # 式(1.5)や式(1.7)にあるh(t-1), c(t-1)
        prevh = H[t-1] if t > 0 else h0
        prevc = C[t-1] if t > 0 else c0
        x = X[t]
        ###############START CODE HERE###############
        ones = np.ones((batch_size, 1))
        x_h = np.hstack([x, prevh, ones])
        gates_h = np.dot(x_h, W)
        i_h, o_h, f_h, a_h =np.split(gates_h, 4, axis = 1)
        
        i = sigmoid(i_h)
        o = sigmoid(o_h)
        f = sigmoid(f_h)
        a = tanh(a_h)
        
        c = i * a + f * prevc
        ct = tanh(c)       
        h = o * ct
        ################END CODE HERE################
        # 状態を保存
        X_h[t] = x_h
        Gates[t] = np.concatenate([i, o, f, a], axis=1)
        C[t] = c
        Ct[t] = ct
        H[t] = h

    cache = {}
    cache['W'] = W
    cache['X_h'] = X_h
    cache['Gates'] = Gates
    cache['C'] = C
    cache['Ct'] = Ct
    cache['H'] = H
    cache['c0'] = c0
    cache['h0'] = h0

    return H, cache, C[t], H[t]

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [7]:
%%bash
./validation_client.py dele-02 5 21 Step5_07.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



LSTMの順伝播を実行してみましょう。

In [8]:
rng = np.random.RandomState(1234)

n_input, n_hidden = 2, 3
W = LSTM_init_parameters(n_input,n_hidden)
batch_size, length = 3, 12

sample_X = rng.randn(length, batch_size, n_input)
H, cache_LSTM, c_last, H_last  = LSTM_forward(sample_X, W)
print('H_last = \n', H_last)

H_last = 
 [[ 0.06783421  0.11366456  0.06664872]
 [ 0.08703627 -0.07033305  0.00861095]
 [-0.0500864   0.01666135  0.01540065]]


**Expected Output**

H_last = 

 [[ 0.06783421  0.11366456  0.06664872]
 
 [ 0.08703627 -0.07033305  0.00861095]
 
 [-0.0500864   0.01666135  0.01540065]]


### LSTMの逆伝播

ここにはLSTMの逆伝播の関数があります。LSTMの逆伝播自体はこれまでのRNNとさほど変わりはないのですが、コードにするとややわかりにくくなるので、実装はしません。ただ、一度この関数内でどのような計算を行っているのか見てみてください。

関数`LSTM_backward`は４つの引数があります。
- `dprev`: 上の層からの勾配
- `cache`: 順伝播で保存しておいたキャッシュ
- `dcn, dhn`: CECや隠れ状態に勾配がすでにある場合に使用

In [9]:
def LSTM_backward(dprev, cache, dcn = None, dhn = None): 
    # 順伝播の時のキャッシュを取得
    W = cache['W']
    X_h = cache['X_h']
    Gates = cache['Gates']
    C = cache['C']
    Ct = cache['Ct']
    H = cache['H']
    c0 = cache['c0']
    h0 = cache['h0']
    
    # パラメータを取得
    length, batch_size, n_hidden = H.shape
    n_input = W.shape[0] - n_hidden - 1

    # 勾配を格納するための配列
    dW = np.zeros_like(W)
    dC = np.zeros_like(C)
    dX = np.zeros((length,batch_size,n_input))
    
    # dprevを書き換えないようにコピー
    dH = dprev.copy()
    # dcn, dhnがある場合は予め勾配を足しておく
    if dcn is not None: dC[-1] += dcn.copy()
    if dhn is not None: dH[-1] += dhn.copy()
    
    for t in reversed(range(length)):
        # ゲートなどを取得
        i, o, f, a = np.hsplit(Gates[t], 4) 
        
        # dL/do = dL/dh * dh/do : batckprop through output-gate
        do = Ct[t] * dH[t]
        # dL/dc = dL/dh * dh/dct * dct/dc = dh[t] * o * (1-tanhCt**2)
        dC[t] += dtanh(Ct[t]) * (o * dH[t])
        
        if t > 0:
            # dL/df = dL/dc * dc/df : backprop through forget_gate
            df = C[t-1] * dC[t]
            # dL/dc(t-1) = dL/dc * dc/dc(t-1)
            dC[t-1] += f * dC[t]
        else:
            # dL/df = dL/dc * dc/df : backprop through forget_gate
            df = c0 * dC[t]
            # dL/dc(t-1) = dL/dc * dc/df
            dc0 = f * dC[t]
            
        # dL/di = dL/dc * dc/da
        di = a * dC[t]        
        # dL/da = dL/dc * dc/da
        da = i * dC[t]

        # dL/gates = dL/dGates * dGates/gates (gates=(i_hat, o_hat, f_hat))
        di_hat = dsigmoid(i) * di
        do_hat = dsigmoid(o) * do
        df_hat = dsigmoid(f) * df
        # dL/a_hat = dL/da * da/a_hat
        da_hat = dtanh(a) * da
        
        dgates_h = np.concatenate((di_hat, do_hat, df_hat, da_hat), axis=1)

        # dL/dW = dL/dgates * dgates/dW
        dW += np.dot(X_h[t].T, dgates_h)
        
        # dL/dh_in = dL/dgates_h * dgates_h/dh_in
        dx_h = np.dot(dgates_h, W.T)

        # dL/dX
        dX[t] = dx_h[:,:n_input]
        if t > 0:
            # dL/dh(t-1)
            dH[t-1] += dx_h[:,n_input:-1]
        else:
            dh0 = dx_h[:,n_input:-1]

    return dX, dW, dc0, dh0

LSTMの逆伝播を実行してみましょう。

In [10]:
rng = np.random.RandomState(1234)
wrand = rng.randn(*H.shape)
dH = wrand
dX, dparams, dc0, dh0 = LSTM_backward(dH, cache_LSTM)
print('dh0 = \n',dh0)

dh0 = 
 [[ 0.12472003  0.12491856  0.04649688]
 [-0.01681211  0.19066373 -0.10475641]
 [-0.16024844  0.06606487  0.17958243]]


**Expected Output**

dh0 = 

[[ 0.12472003  0.12491856  0.04649688]

[-0.01681211  0.19066373 -0.10475641]

[-0.16024844  0.06606487  0.17958243]]

### LSTMの勾配チェック

**【課題３】**　LSTMの逆伝播をチェックするコードを作成します。

NNやCNNそしてRNNにおいて勾配計算を実装する際に、コードエラーが、なくて一見学習も進んでいるように見えても実は勾配計算でミスを犯している、などということは多々あります。そこで、連鎖率を使って計算した勾配と数値微分の値が一致するか確認する必要があります。ここでは実際にこれらが一致しているか勾配チェックを通して確かめます。

数値微分とは、
$$
\frac{dJ(\theta)}{d\theta} =  \lim_{\epsilon\to 0} 
\frac{J(\theta+\epsilon) - J(\theta-\epsilon)}{2\epsilon}\tag{1.9}
$$
で計算できます。ここではLSTMのパラメータの値１つずつ微小変化を加え、数値微分を求め実装した勾配計算との誤差を見ます。ここで注意すべき点はパラメータ$\boldsymbol{W_i}$に微小変化を加えるのではなく、$\boldsymbol{W_i}$の中の１つの値に微小変化を加えます。$\boldsymbol{W_i}$が3×4の行列だとしたら１２回数値微分を行い、その都度勾配をチェックします。コードではこれが２つ目のforループに当たります。

上式を使って数値微分を求めたら最後に実装した勾配との差を見ます。ただこの時に数値微分の値そのものが大きいあるいは小さいこともあるので、比較ができるように誤差を数値微分と勾配を足し合わせた値で割ります。コードでは`rel_error`に当たります。この値が$10^{-10}$よりも小さければ勾配計算はほとんどの場合あっています。値が大きい場合は勾配計算が間違っている（今回は実装していないのでありえません）か、勾配チェックのコードが間違っています。

In [11]:
#Coursedele-02 Step5 QuestionNumber22 ffbb1c4eec11b54c652979689f5d2594
rng = np.random.RandomState(1234)
epsilon = 1e-5

# パラメータを初期化
n_input, n_hidden = 2, 3
batch_size, length = 3, 12
params = LSTM_init_parameters(n_input, n_hidden)
X = rng.randn(length,batch_size,n_input)
c0 = rng.randn(batch_size,n_hidden)
h0 = rng.randn(batch_size,n_hidden)

# 順伝播（*_とするとそれ以降の返り値と１つのタプルに格納、返り値を破棄する場合に便利）
H, cache, *_  = LSTM_forward(X, params, c0=c0, h0=h0)
# 順伝播の出力とrandomYをかけて足したものを損失とする
randomY = rng.randn(*H.shape) # 正解ラベル的なもの
loss = np.sum(H * randomY)
# dloss/dH = randomY
dH = randomY
# 逆伝播
dX, dW, dc0, dh0 = LSTM_backward(dH, cache)

variables = (X, params, c0, h0)
grads_analytic = (dX, dW, dc0, dh0)
names = ('dX', 'dW', 'dc0', 'dh0')
for j in range(len(variables)):
    mat = variables[j]
    dmat = grads_analytic[j]
    name = names[j]
    max_rel_error = 0
    for i in range(mat.size):
        ###############START CODE HERE###############
        # theta + epsilon
        mat.flat[i] += epsilon
        # 順伝播を計算
        H, *_ = LSTM_forward(X, params, c0=c0, h0=h0)
        loss1 = np.sum(H * randomY)

        # theta - epsilon、一つ前で+epsilonしているのでここでは-epsilon*2とする。
        mat.flat[i] -= epsilon * 2
        H, *_ = LSTM_forward(X, params, c0=c0, h0=h0)
        loss2 = np.sum(H * randomY)

        # 元に戻す
        mat.flat[i] += epsilon
        
        # 数値微分
        grad_numerical = (loss1 - loss2) / (2 * epsilon)
        ################END CODE HERE################
        
        # 解析的に求めた微分
        grad_analytic = dmat.flat[i]
        
        # gradそのものが大きいあるいは小さい可能性があるので、相対的な誤差を計算
        rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
    if rel_error > max_rel_error: # 最大の誤差を保存
        max_rel_error = rel_error
    print('{}: {:.3e}'.format(name, max_rel_error))

dX: 2.850e-11
dW: 1.376e-11
dc0: 6.193e-11
dh0: 1.201e-11


** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [12]:
%%bash
./validation_client.py dele-02 5 22 Step5_07.ipynb api.internal.zero2one.jp

ExcessiveAccess: Wait for 10 minutes



## GRUの実装

GRUは以下の式計算できます。ここで$\boldsymbol{z}(t)$は更新ゲート、$\boldsymbol{r}(t)$はリセットゲートです。LSTMとは違いゲートは２つしかありません。なので、パラメータもLSTMの１２個と比べて９個少なくなります。

$
\boldsymbol{z}(t)=\sigma(\boldsymbol{W_z}\boldsymbol{x}(t)+\boldsymbol{U_z}\boldsymbol{h}(t-1)+\boldsymbol{b_z}) \tag{2.1}
$

$
\boldsymbol{r}(t)=\sigma(\boldsymbol{W_r}\boldsymbol{x}(t)+\boldsymbol{U_r}\boldsymbol{h}(t-1)+\boldsymbol{b_r}) \tag{2.2}
$

$
\boldsymbol{\tilde{h}}(t)=f(\boldsymbol{W_h}\boldsymbol{x}(t) + \boldsymbol{U_h}(\boldsymbol{r}(t)\odot \boldsymbol{h}(t-1))+\boldsymbol{b_h}) \tag{2.3}
$
$
%\boldsymbol{h}(t)=\boldsymbol{z}(t)\odot\boldsymbol{h}(t-1) + (1-\boldsymbol{z}(t))\odot\boldsymbol{\tilde{h}}(t) \tag{2.4}
$
$
\boldsymbol{h}(t)=(1-\boldsymbol{z}(t))\odot\boldsymbol{h}(t-1) + \boldsymbol{z}(t)\odot\boldsymbol{\tilde{h}}(t) \tag{2.4}
$

### GRUのパラメータの初期化

** 【課題４】 **　GRUの順伝播を実装します。


まずはGRUのパラメータを初期化します。式(2.1)と式(2.2)はまとめて計算することはできるのですが、今回は可読性を優先して、LSTMの時はまとめて初期化しましたが、ここでは別々に初期化します。

関数`GRU_init_parameters`の引数はLSTMと同様に
- n_input: 入力の次元
- n_hidden: 隠れ層のユニット数
です。


In [47]:
#Coursedele-02 Step5 QuestionNumber23 233106a68106cadf77d0d19fd8a3fef3
def GRU_init_parameters(n_input, n_hidden):
    ###############START CODE HERE###############
    Wr = glorot_uniform((n_input, n_hidden))
    Wz = glorot_uniform((n_input, n_hidden))
    Wh = glorot_uniform((n_input, n_hidden))
    Ur = glorot_uniform((n_hidden, n_hidden))
    Uz = glorot_uniform((n_hidden, n_hidden))
    Uh = glorot_uniform((n_hidden, n_hidden))
    br = np.zeros((n_hidden))
    bz = np.zeros((n_hidden))
    bh = np.zeros((n_hidden))
    ################END CODE HERE################
    W = (Wr, Wz, Wh, Ur, Uz, Uh) 
    b = (br, bz, bh)
    return (W, b)

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [48]:
%%bash
./validation_client.py dele-02 5 23 Step5_07.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



### GRUの順伝播の実装

** 【課題５】 **　次にGRUの順伝播を実装します。

関数`GRU_forward`は３つの引数があります。
- X: 入力（データ長、ミニバッチサイズ、入力サイズ）
- W: `LSTM_init_parameters`から得られる重み
- h0: 隠れ層の値（なければゼロで初期化）

ここで実装していただく箇所は式(2.1)~(2.4)に対応する箇所です。

In [56]:
#Coursedele-02 Step5 QuestionNumber24 c07ed3ce024b9e7a572750e8a33888c5
def GRU_forward(X, params, h0 = None):
    Wr, Wz, Wh, Ur, Uz, Uh = params[0]
    br, bz, bh = params[1]
    length, batch_size, n_input = X.shape
    n_input = Wr.shape[0]
    n_hidden = Ur.shape[0]
    
    if h0 is None: h0 = np.zeros((batch_size, n_hidden))
        
    R = np.zeros((length, batch_size, n_hidden))
    Z = np.zeros((length, batch_size, n_hidden))
    H_hat = np.zeros((length, batch_size, n_hidden))
    H = np.zeros((length, batch_size, n_hidden))
    
    for t in range(length):
        h_prev = H[t-1] if t > 0  else h0
        x = X[t]
        ###############START CODE HERE###############
        # 式(2.1)
        z = sigmoid(np.dot(x, Wz) + np.dot(H[t-1], Uz) + bz)
        # 式(2.2)
        r = sigmoid(np.dot(x, Wr) + np.dot(H[t-1], Ur) + br)
        # 式(2.3)
        h_hat = tanh(np.dot(x, Wh) + np.dot(r * H[t-1], Uh) + bh)
        # 式(2.4)
        h_next = (1 - z) * H[t-1] + z * h_hat
        ################END CODE HERE################
        Z[t] = z
        R[t] = r
        H_hat[t] = h_hat
        H[t] = h_next

    cache = {}
    cache['params'] = params
    cache['X'] = X
    cache['Z'] = Z
    cache['R'] = R
    cache['H_hat'] = H_hat
    cache['H'] = H
    cache['h0'] = h0

    return H, cache, H[t]

** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [57]:
%%bash
./validation_client.py dele-02 5 24 Step5_07.ipynb api.internal.zero2one.jp

Congratulations!
We give you 10 points out of 10 points.



GRUの順伝播を実行してみましょう。

In [58]:
rng = np.random.RandomState(1234)

n_input, n_hidden = 2, 3
params = GRU_init_parameters(n_input,n_hidden)
batch_size, length = 3, 12

sample_X = rng.randn(length, batch_size, n_input)
H, cache_GRU, H_last = GRU_forward(sample_X, params)
print('H_last = \n', H_last)

H_last = 
 [[ 0.2176545  -0.00864095  0.4214221 ]
 [ 0.12158509  0.34806795 -0.02168339]
 [ 0.47930925  0.23608282  0.15999168]]


**Expected Output**

H_last = 

 [[ 0.2176545  -0.00864095  0.4214221 ]
 
 [ 0.12158509  0.34806795 -0.02168339]
 
 [ 0.47930925  0.23608282  0.15999168]]


### GRUの逆伝播

ここにはGRUの逆伝播の関数があります。ここでも逆伝播は実装していただきません。

関数`GRU_backward`は４つの引数があります。
- `dprev`: 上の層からの勾配
- `cache`: 順伝播で保存しておいたキャッシュ
- `dhn`: 隠れ状態に勾配がすでにある場合に使用

In [59]:
def GRU_backward(dprev, cache, dhn = None):
    params = cache['params']
    X = cache['X']
    R = cache['R']
    Z = cache['Z']
    H_hat = cache['H_hat']
    H = cache['H']
    h0 = cache['h0'] 
    
    Wr, Wz, Wh, Ur, Uz, Uh = params[0]
    br, bz, bh = params[1]
    length, batch_size, n_input = X.shape
    n_input = Wr.shape[0]
    n_hidden = Ur.shape[0]
    
    dW_ih = np.zeros((n_input, 3*n_hidden))
    dW_hh = np.zeros((n_hidden, 3*n_hidden))
    dbias = np.zeros((n_hidden*3))
    dWr, dWz, dWh = np.split(dW_ih, 3, axis=1)
    dUr, dUz, dUh = np.split(dW_hh, 3, axis=1)
    dbr, dbz, dbh = np.split(dbias, 3, axis=0)
    dX = np.zeros((length,batch_size,n_input))
    dprev = np.copy(dprev)
    for t in reversed(range(length)):
        z = Z[t]
        dh = dprev[t]
        h_prev = H[t-1] if t > 0 else h0
        h_hat = H_hat[t]
        r = R[t]
        x = X[t]
        ### 勾配計算 ###
        dz = (h_hat-h_prev) * dh
        dh_prev = (1-z) * dh
        dh_hat = z * dh
        
        dh_hat_h = dtanh(h_hat) * dh_hat
        dWh += np.dot(x.T, dh_hat_h)
        dx  = np.dot(dh_hat_h, Wh.T)
        dUh += np.dot((r*h_prev).T, dh_hat_h)
        dr = h_prev * np.dot(dh_hat_h, Uh.T)
        dh_prev += r * np.dot(dh_hat_h, Uh.T)
        dbh += np.sum(dh_hat_h, axis=0)
        
        dr_h = dsigmoid(r) * dr
        dWr += np.dot(x.T, dr_h)
        dx += np.dot(dr_h, Wr.T)
        dUr += np.dot(h_prev.T, dr_h)
        dh_prev += np.dot(dr_h, Ur.T)
        dbr += np.sum(dr_h, axis=0)
                      
        dt = dsigmoid(z) * dz
        dWz += np.dot(x.T, dt)
        dx += np.dot(dt, Wz.T)
        dUz += np.dot(h_prev.T, dt)
        dh_prev += np.dot(dt, Uz.T)
        dbz += np.sum(dt, axis=0)

        dX[t] = dx
        if t > 0:
            dprev[t-1] += dh_prev
        else:
            dh0 = dh_prev
        ############
    dparams = (dWr, dWz, dWh, dUr, dUz, dUh, dbr, dbz, dbh)
    return dX, dparams, dh0

In [60]:
rng = np.random.RandomState(1234)
wrand = rng.randn(*H.shape)
dH = wrand
dX, dparams,dh0 = GRU_backward(dH, cache_GRU)
print('dh0 = \n',dh0)

dh0 = 
 [[-1.29291966 -1.03813569  1.53903238]
 [ 0.40343046 -1.57202558  0.64777347]
 [-0.03634266 -0.18345031  0.05864957]]


**Expected Output**

dh0 = 

 [[-1.29291966 -1.03813569  1.53903238]
 
 [ 0.40343046 -1.57202558  0.64777347]
 
 [-0.03634266 -0.18345031  0.05864957]]

### GRUの勾配チェック

** 【課題６】 **　GRUの逆伝播をチェックするコードを作成します。

LSTMの時とほぼ同じですが、もう一度勾配チェックのコードを実装してください。実装していただく箇所でLSTMの時と異なる箇所はLSTM_forwardとその引数だけです。

In [61]:
#Coursedele-02 Step5 QuestionNumber25 255fd4ffe650359e8d9571a1e13b41d0
rng = np.random.RandomState(1234)
epsilon = 1e-5

# パラメータを初期化
n_input, n_hidden = 2, 3
batch_size, length = 3, 12
params = GRU_init_parameters(n_input, n_hidden)
X = rng.randn(length,batch_size,n_input)
h0 = rng.randn(batch_size,n_hidden)

# 順伝播
H, cache, Ht = GRU_forward(X, params, h0=h0)
# 順伝播の出力とrandomYをかけて足したものを損失とする
randomY = rng.randn(*H.shape)
loss = np.sum(H * randomY)
# dloss/dH = randomY
dH = randomY
# 逆伝播
dX, dparams, dh0 = GRU_backward(dH, cache)

variables = (X,) + params[0] + params[1] + (h0,)
grads_analytic = (dX, ) + dparams + (dh0,)
names = ['dX', 'dWr', 'dWz', 'dWh', 
         'dUr', 'dUz', 'dUh', 
         'dbr', 'dbz', 'dbh', 'dh0']
for j in range(len(variables)):
    mat = variables[j]
    dmat = grads_analytic[j]
    name = names[j]
    max_rel_error = 0
    for i in range(mat.size):
        ###############START CODE HERE###############
        # theta + epsilon
        mat.flat[i] += epsilon
        H, *_ = GRU_forward(X, params, h0=h0)
        loss1 = np.sum(H * randomY)

        # theta - epsilon、一つ前で+epsilonしているのでここでは-epsilon*2とする。
        mat.flat[i] -= 2 * epsilon
        H, *_ = GRU_forward(X, params, h0=h0)
        loss2 = np.sum(H * randomY)

        # 元に戻す
        mat.flat[i] += epsilon
        # 数値微分
        grad_numerical = (loss1 - loss2) / (2 * epsilon)
        ################END CODE HERE################
        
        # 解析的に求めた微分
        grad_analytic = dmat.flat[i]
        
        # gradそのものが大きいあるいは小さい可能性があるので、相対的な誤差を計算
        rel_error = abs(grad_analytic - grad_numerical) / abs(grad_numerical + grad_analytic)
    if rel_error > max_rel_error: # 最大の誤差を保存
        max_rel_error = rel_error
    print('{}: {:.3e}'.format(name, max_rel_error))

dX: 2.199e-11
dWr: 7.964e-02
dWz: 2.711e-03
dWh: 1.314e-10
dUr: 1.149e+00
dUz: 2.083e-01
dUh: 2.388e-02
dbr: 2.760e-01
dbz: 5.562e-02
dbh: 5.521e-12
dh0: 1.000e+00


** ファイルを保存後 **、次のセルを実行（Shift+Enter）で採点を行います。

In [62]:
%%bash
./validation_client.py dele-02 5 25 Step5_07.ipynb api.internal.zero2one.jp

dX: 6.062e-12
dWr: 2.266e-11
dWz: 1.337e-12
dWh: 5.007e-11
dUr: 1.858e-10
dUz: 3.232e-11
dUh: 1.496e-10
dbr: 1.032e-10
dbz: 4.520e-11
dbh: 2.992e-11
dh0: 1.186e-11
Congratulations!
We give you 10 points out of 10 points.

